Make sure Python and Jupyter are set up locally first (and if necessary relaunch VSCode). There is a `shell.nix` that does this for you if you have Nix installed, but it won't work with some of the Spring AI library dependencies (the ones that have native code extensions), so it's better to use a virtual env created from the global Python.

```bash
$ python3 -m venv .venv
$ . .venv/bin/activate
$ pip install jupyter ipykernel notebook
```

Then, select the Python interpreter in the top right corner of the VSCode editor - it should be `java (Rapaio/j!)`. If that is not available, install it with `jbang install-kernel@jupyter-java rapaio`. Make sure to get the latest version of `jbang` and also edit the `kernel.json` file in `~/.local/share/jupyter/kernels` to upgrade to version 2.0.0 and Java 22.

```json
{
  "argv" : [
    "/home/dsyer/.sdkman/candidates/jbang/current/bin/jbang",
    "--java",
    "22",
...
    "io.github.padreati:rapaio-jupyter-kernel:2.0.0@fatjar",
    "{connection_file}"
  ],
...
}
```

In [1]:
%classpath target/classes/

Add /home/dsyer/dev/scratch/enhance-llm/target/classes to classpath


In [2]:
%%jars
target/lib

Found 103 jar files.
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jackson-datatype-jdk8-2.17.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/typetools-0.6.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/spring-boot-starter-logging-3.3.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jackson-core-2.17.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/logback-classic-1.5.6.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/spring-ai-core-1.0.0-SNAPSHOT.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/hamcrest-core-2.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jackson-databind-2.17.2.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jtokkit-1.1.0.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/spring-ai-ollama-1.0.0-SNAPSHOT.jar to classpath
Add /home/dsyer/dev/scratch/enhance-llm/target/lib/jsonschem

In [3]:
import org.springframework.boot.logging.*;
// Make sure Spring Boot is using the Java Logging System not Logback
System.setProperty(LoggingSystem.SYSTEM_PROPERTY, "org.springframework.boot.logging.java.JavaLoggingSystem");

Run the vector store in the background:

```bash
$ docker run -it --rm --name chroma -p 8000:8000 ghcr.io/chroma-core/chroma:0.4.15
```

and then the app will be able to connect to it when it starts up.

Models prepared in onxx format:

```bash
$ pip install optimum onnx onnxruntime
$ optimum-cli export onnx --model sentence-transformers/multi-qa-MiniLM-L6-cos-v1 onnx-output
```

In [10]:
%%bash
ls -l onnx-output

total 102052
-rw-r--r-- 1 dsyer dsyer      697 Aug  5 09:55 config.json
-rw-r--r-- 1 dsyer dsyer 13085339 Aug  5 09:59 databricks-dolly-15k.jsonl
-rw-r--r-- 1 dsyer dsyer 90447733 Aug  5 09:55 model.onnx
-rw-r--r-- 1 dsyer dsyer      695 Aug  5 09:55 special_tokens_map.json
-rw-r--r-- 1 dsyer dsyer   711661 Aug  5 09:55 tokenizer.json
-rw-r--r-- 1 dsyer dsyer     1433 Aug  5 09:55 tokenizer_config.json
-rw-r--r-- 1 dsyer dsyer   231508 Aug  5 09:55 vocab.txt


Make sure ollama is running in the background and the model is available before we start the app:

```bash
$ ollama serve
$ ollama pull mistral
$ ollama pull albertogg/multi-qa-minilm-l6-cos-v1
```

In [6]:
import org.springframework.boot.SpringApplication;
import com.example.DemoApplication;
var app = SpringApplication.run(DemoApplication.class, new String[] {});

[Engine-thread-2] INFO org.springframework.boot.devtools.env.DevToolsPropertyDefaultsPostProcessor - For additional web related logging consider setting the 'logging.level.web' property to 'DEBUG'



  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/

 :: Spring Boot ::                (v3.3.2)



[Engine-thread-2] INFO org.springframework.boot.SpringApplication - Starting application using Java 22.0.2 with PID 1885992 (started by dsyer in /home/dsyer/dev/scratch/enhance-llm)
[Engine-thread-2] INFO org.springframework.boot.SpringApplication - No active profile set, falling back to 1 default profile: "default"
[Engine-thread-2] INFO org.springframework.boot.web.embedded.tomcat.TomcatWebServer - Tomcat initialized with port 8080 (http)
[Engine-thread-2] INFO org.springframework.boot.web.servlet.context.ServletWebServerApplicationContext - Root WebApplicationContext: initialization completed in 397 ms
[Engine-thread-2] INFO org.springframework.boot.web.embedded.tomcat.TomcatWebServer - Tomcat started on port 8080 (http) with context path '/'
[Engine-thread-2] INFO org.springframework.boot.SpringApplication - Started application in 0.723 seconds (process running for 233.472)


In [10]:
import org.springframework.ai.embedding.EmbeddingModel;
var model = app.getBean(EmbeddingModel.class);
model.embed("Hello World");

[Engine-thread-3] WARN org.springframework.ai.ollama.api.OllamaApi - [404] Not Found - 404 page not found


EvalException: [404] Not Found - 404 page not found

In [6]:
model.embed("Hello London");

[0.18317344784736633, -0.03303219750523567, 0.509656548500061, -0.0716996043920517, -0.29321762919425964, -0.3439341187477112, 0.5183614492416382, -0.5363336801528931, -0.3582697808742523, -0.2888084053993225, -0.08874968439340591, -0.15257827937602997, -0.15399040281772614, -0.180519700050354, -0.008928481489419937, -0.09243278950452805, 0.30338582396507263, -0.5649009346961975, -0.7345456480979919, -0.18324366211891174, 0.11387030780315399, 0.2951998710632324, -0.03293706476688385, 0.3965093493461609, -0.39608675241470337, 0.028525378555059433, 0.3334903419017792, 0.4080807566642761, 0.08179500699043274, -0.13933022320270538, 0.2384822964668274, 0.029904760420322418, 0.3230598568916321, 0.023913010954856873, 0.21026787161827087, 0.5828791856765747, -0.24053677916526794, -0.24509379267692566, -0.10948795080184937, -0.01229856163263321, 0.23731942474842072, -0.5487273335456848, -0.04797591269016266, -0.2077256739139557, 0.3372061252593994, 0.0646229237318039, 0.20259717106819153, 0.248

In [7]:
import com.fasterxml.jackson.databind.ObjectMapper;
var mapper = app.getBean(ObjectMapper.class)

In [8]:
var reader = mapper.readerFor(Map.class)

In [11]:
reader.readValues(new File("./data/lines.jsonl")).forEachRemaining(line -> System.out.println(((Map)line).get("instruction")));

How to make a cup of tea?
What is the capital of France?
What is the capital of Germany?
What is the capital of Italy?
What is the capital of Spain?
What is the capital of Portugal?
What is the capital of Greece?
What is the capital of Turkey?
What is the capital of Egypt?
What is the capital of South Africa?
What is the capital of Nigeria?
What is the capital of Kenya?
What is the capital of India?
What is the capital of China?
What is the capital of Japan?
What is the capital of Australia?
What is the capital of New Zealand?
What is the capital of Canada?
What is the capital of the United States?
What is the capital of Brazil?
What is the capital of Argentina?
What is the capital of Chile?
What is the capital of Peru?


In [9]:
import org.springframework.ai.document.Document;
import org.springframework.ai.vectorstore.VectorStore;

var store = app.getBean(VectorStore.class);

The dataset is stored in huggingface in `git-lfs` format, so you need to install `git-lfs` and then run `git lfs install && git lfs checkout` in the repo to download the data. Or download it manually from the [Hugging Face website](https://huggingface.co/datasets/databricks-dolly-15k). Or use the HugingFace CLI:

```bash
$ huggingface-cli download --local-dir onnx-output --repo-type dataset databricks/databricks-dolly-15k databricks-dolly-15k.jsonl
```

In [10]:
import org.springframework.util.StringUtils;
import org.springframework.ai.transformer.splitter.TokenTextSplitter;
List<Document> list = new ArrayList<>();
TokenTextSplitter splitter = new TokenTextSplitter(200, 50, 5, 400, true);
reader.readValues(new File("./onnx-output/databricks-dolly-15k.jsonl")).forEachRemaining(line -> {
	Map map = (Map) line;
	if (!"closed_qa".equals(map.get("category"))) {
		return;
	}
	String context = ((String) map.get("context")).trim();
	if (StringUtils.hasText(context)) {
		Document doc = new Document(context);
		List<Document> splitDocs = splitter.split(doc);
		list.addAll(splitDocs);
	}
});
store.add(list);

[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 3 chunks.
[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.
[Engine-thread-0] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 4 chunks.
[Engine-thread-0] INFO org.springframework.ai.tr

In [11]:
store.similaritySearch("When was Tomoaki Komorida born?")

[Document{id='11cc0be0-bfc3-48d8-a50f-4d677009c94b', metadata={distance=0.45460922}, content='Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer.', media=[]}, Document{id='3dacca26-b984-42f5-85f5-ea19f11f8cc1', metadata={distance=0.60816765}, content='These were made in the period between the 3rd century BCE and the 1st century AD.

In 477

In [13]:
import org.springframework.ai.chat.model.ChatModel;
var chat = app.getBean(ChatModel.class);

In [14]:
chat.call("When was Tomoaki Komorida born?")

 I'm sorry for any confusion, but the question seems to be missing a specific context. There are several people named Tomoaki Komorada or Komorida across various fields such as manga, baseball, and sumo wrestling. Without specifying which one you are referring to, it is difficult to provide an accurate birthdate. If you could provide more details, I would be happy to help!

In [23]:
import org.springframework.ai.chat.client.ChatClient;
import org.springframework.ai.chat.client.advisor.QuestionAnswerAdvisor;
import org.springframework.ai.vectorstore.SearchRequest;
var builder = app.getBean(ChatClient.Builder.class);
var response = builder.build().prompt()
.advisors(new QuestionAnswerAdvisor(store, SearchRequest.defaults()))
.user("When was Tomoaki Komorida born?")
.call();

In [24]:
response.content()

 Based on the provided context, it does not contain any information about when Tomoaki Komorida was born. Therefore, I cannot provide an accurate answer to your question.

In [30]:
splitter.split(new Document("Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer. In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played as a regular player and the club was promoted to J2 in 2008. Although he did not play as much, he still played in many matches. In 2010, he moved to Indonesia and joined Persela Lamongan. In July 2010, he returned to Japan and joined the J2 club Giravanz Kitakyushu. He played often as a defensive midfielder and center back until 2012 when he retired."))

[Engine-thread-5] INFO org.springframework.ai.transformer.splitter.TextSplitter - Splitting up document into 2 chunks.


[Document{id='dc6b45a0-5014-4231-8b1a-654e1dd80999', metadata={}, content='Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer.', media=[]}, Document{id='38c4969b-5cb7-49f3-a360-3894caf38bc6', metadata={}, content='In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played